In [60]:
#carregando bibliotecas
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.preprocessing import StandardScaler,OneHotEncoder,MinMaxScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel

In [19]:
#carrega dados de mercado
df_market = pd.read_csv('../data/estaticos_market.csv')

In [3]:
#carrega portfólio
df_portfolio = pd.read_csv('../data/estaticos_portfolio1.csv')

In [20]:
#coloca id como índice
df_market = df_market.set_index('id')

In [65]:
#selecte boolean features
def select_by_variance(df,threshold,var_type):
    sel_bool = VarianceThreshold(threshold=threshold)
    fitted_model = sel.fit(df.select_dtypes(include=var_type))
    features = df.select_dtypes(include=var_type).columns[fitted_model.get_support()]
    return features

def preprocessing(df,col_group,nan_resolve):
    info_empresa = ['de_indicador_telefone',
    'de_nivel_atividade',
    'de_ramo',
    'fl_email',
    'fl_mei',
    'fl_telefone',
    'idade_empresa_anos',
    'natureza_juridica_macro',
    'nm_divisao',
    'qt_filiais',
    'sg_uf']
    print('--Criando dataframe com as colunas--')
    df_empresa = df[col_group]

    if nan_resolve == 'median':
        imputer = SimpleImputer(missing_values=np.nan,strategy='median')
        imputed = imputer.fit_transform(df_empresa)
        df_empresa = pd.DataFrame(data=imputed,columns=df_empresa.columns)
    else:
        pass

    print('--Aplicando OneHotEnconding--')
    df_encoded = pd.get_dummies(df_empresa)

    print('--Aplicando MinMaxScaler--')
    minmax = MinMaxScaler()
    minmax_transformed = minmax.fit_transform(df_encoded)
    df_empresa = pd.DataFrame(data=minmax_transformed,columns=df_encoded.columns)

    print('--split features target--')
    target = df[['de_ramo']]
    features = df_encoded

    print('--Aplicando Árvore--')
    decision_tree = DecisionTreeClassifier(random_state=2321,max_depth=10)
    model_fited = decision_tree.fit(features,target)
    print('--Selecionando Modelo--')
    model = SelectFromModel(model_fited,prefit=True)
    col_features = df_encoded.columns[model.get_support()]
    df_return = df_encoded[col_features]
    return df_return

#função para recomendar leads
def recommender(df_test,df_market,modelo):
    distance, index = modelo.kneighbors(df_test)
    distance = distance.flatten()
    index = index.flatten()
    recomendados = df_market.iloc[index]
    recomendados['distance'] = distance
    recomendados = recomendados.sort_values(by=['distance'],ascending=True)
    return recomendados['id']

def calculate_accuracy(ids_true, ids_pred):
    return len(set(ids_true) & set(ids_pred)) / len(set(ids_true))

## Teste de seleção de features colocando o ramo da empresa como variável alvo

## Qualificação das variáveis
1. As variáveis foram agrupadas em 7 grupos
2. As features serão avaliadas isoladamente dentro de cada grupo
3. Após esta análise ser escolhida uma variável categórica para ser o target
4. Será feita uma seleção de feature utilizando RandomForest. Será utilizado o inverso do índice de Gini como peso para cada variável

In [6]:
#modelos
enc = OneHotEncoder(handle_unknown='ignore')
std = StandardScaler()

In [30]:
#convertendo variaveis object para bool
convert_to_bool = [
'fl_optante_simei',
'fl_optante_simples',
'fl_passivel_iss',
'fl_rm',
'fl_sa',
'fl_simples_irregular',
'fl_spa',
'fl_st_especial',
'fl_telefone',
'fl_veiculo'
]
for col in convert_to_bool:
    df_market[col] = df_market[col].astype(bool)

In [31]:
info_empresa = ['de_indicador_telefone',
'de_nivel_atividade',
'de_ramo',
'fl_email',
'fl_mei',
'fl_telefone',
'idade_empresa_anos',
'natureza_juridica_macro',
'nm_divisao',
'qt_filiais',
'sg_uf']

In [51]:
info_filiais = [
'coligada_mais_antiga_ativa',
'coligada_mais_antiga_baixada',
'coligada_mais_nova_ativa',
'coligada_mais_nova_baixada',
'de_faixa_faturamento_estimado',
'de_faixa_faturamento_estimado_grupo',
'faturamento_est_coligados',
'faturamento_est_coligados_gp',
'idade_maxima_coligadas',
'idade_media_coligadas',
'idade_media_coligadas_ativas',
'idade_media_coligadas_baixadas',
'idade_minima_coligadas',
'max_faturamento_est_coligados',
'max_faturamento_est_coligados_gp',
'max_filiais_coligados',
'max_funcionarios_coligados_gp',
'max_vl_folha_coligados',
'max_vl_folha_coligados_gp',
'media_faturamento_est_coligados',
'media_faturamento_est_coligados_gp',
'media_filiais_coligados',
'media_funcionarios_coligados_gp',
'media_vl_folha_coligados',
'media_vl_folha_coligados_gp',
'min_faturamento_est_coligados',
'min_faturamento_est_coligados_gp',
'min_filiais_coligados',
'min_funcionarios_coligados_gp',
'min_vl_folha_coligados',
'min_vl_folha_coligados_gp',
'qt_coligadas',
'qt_coligados',
'qt_coligados_agropecuaria',
'qt_coligados_atividade_alto',
'qt_coligados_atividade_baixo',
'qt_coligados_atividade_inativo',
'qt_coligados_atividade_medio',
'qt_coligados_atividade_mt_baixo',
'qt_coligados_ativo',
'qt_coligados_baixada',
'qt_coligados_ccivil',
'qt_coligados_centro',
'qt_coligados_comercio',
'qt_coligados_epp',
'qt_coligados_exterior',
'qt_coligados_inapta',
'qt_coligados_industria',
'qt_coligados_ltda',
'qt_coligados_matriz',
'qt_coligados_me',
'qt_coligados_mei',
'qt_coligados_nordeste',
'qt_coligados_norte',
'qt_coligados_nula',
'qt_coligados_sa',
'qt_coligados_serviço',
'qt_coligados_sudeste',
'qt_coligados_sul',
'qt_coligados_suspensa',
'qt_funcionarios_coligados',
'qt_funcionarios_coligados_gp',
'qt_ramos_coligados',
'qt_regioes_coligados',
'qt_socios_coligados',
'qt_ufs_coligados',
'sum_faturamento_estimado_coligadas',
'total_filiais_coligados',
'vl_faturamento_estimado_aux',
'vl_faturamento_estimado_grupo_aux',
'vl_folha_coligados',
'vl_folha_coligados_gp'
]

In [52]:
info_funcionarios = [
'grau_instrucao_macro_analfabeto',
'grau_instrucao_macro_desconhecido',
'grau_instrucao_macro_escolaridade_fundamental',
'grau_instrucao_macro_escolaridade_media',
'grau_instrucao_macro_escolaridade_superior',
'idade_acima_de_58',
'idade_ate_18',
'idade_de_19_a_23',
'idade_de_24_a_28',
'idade_de_29_a_33',
'idade_de_34_a_38',
'idade_de_39_a_43',
'idade_de_44_a_48',
'idade_de_49_a_53',
'idade_de_54_a_58',
'max_meses_servicos',
'max_meses_servicos_all',
'media_meses_servicos',
'media_meses_servicos_all',
'meses_ultima_contratacaco',
'min_meses_servicos',
'min_meses_servicos_all',
'percent_func_genero_fem',
'percent_func_genero_masc',
'qt_admitidos',
'qt_admitidos_12meses',
'qt_desligados',
'qt_desligados_12meses',
'qt_ex_funcionarios',
'qt_funcionarios',
'qt_funcionarios_12meses',
'qt_funcionarios_24meses',
'qt_funcionarios_grupo',
'total',
'tx_crescimento_12meses',
'tx_crescimento_24meses',
'tx_rotatividade'
]

In [53]:
info_ibge = [
'empsetorcensitariofaixarendapopulacao',
'nm_micro_regiao'    
]

In [54]:
info_socios = [
'idade_maxima_socios',
'idade_media_socios',
'idade_minima_socios',
'qt_alteracao_socio_180d',
'qt_alteracao_socio_365d',
'qt_alteracao_socio_90d',
'qt_alteracao_socio_total',
'qt_socios',
'qt_socios_feminino',
'qt_socios_masculino',
'qt_socios_pep',
'qt_socios_pf',
'qt_socios_pj',
'qt_socios_pj_ativos',
'qt_socios_pj_baixados',
'qt_socios_pj_inaptos',
'qt_socios_pj_nulos',
'qt_socios_pj_suspensos',
'qt_socios_st_regular',
'qt_socios_st_suspensa',
'vl_idade_maxima_socios_pj',
'vl_idade_media_socios_pj',
'vl_idade_minima_socios_pj',    
]

In [55]:
info_transporte = [
'fl_antt',
'fl_optante_simples',
'fl_spa',
'fl_veiculo',
'qt_art',
'vl_frota',
'vl_potenc_cons_oleo_gas',
'vl_total_tancagem',
'vl_total_tancagem_grupo',
'vl_total_veiculos_antt',
'vl_total_veiculos_antt_grupo',
'vl_total_veiculos_leves',
'vl_total_veiculos_leves_grupo',
'vl_total_veiculos_pesados',
'vl_total_veiculos_pesados_grupo'    
]

In [56]:
info_tributaria = [
'de_saude_rescencia',
'de_saude_tributaria',
'fl_optante_simei',
'fl_passivel_iss',
'fl_simples_irregular',
'nu_meses_rescencia'
]

In [47]:
df_empresa = preprocessing(df_market,info_empresa,'none')

--Criando dataframe com as colunas--
--Aplicando OneHotEnconding--
--Aplicando MinMaxScaler--
--split features target--
--Aplicando Árvore--
--Selecionando Modelo--


In [66]:
df_funcionarios = preprocessing(df_market,info_funcionarios,'median')

--Criando dataframe com as colunas--


ValueError: Shape of passed values is (462298, 36), indices imply (462298, 37)

### Teste do KNN somente com as variáveis selecionadas para a empresa

In [219]:
df_features_enconding_std['id'] = df_market.index
df_portfolio1 = pd.read_csv('../data/estaticos_portfolio2.csv')
df_portfolio1 = df_features_enconding_std[df_features_enconding_std['id'].isin(df_portfolio['id'])]

In [50]:
#treino do modelo
model_knn = NearestNeighbors(metric='cosine',algorithm='brute',n_neighbors=7,n_jobs=-1)
model_knn = model_knn.fit(df_features_enconding_std)

NameError: name 'df_features_enconding_std' is not defined

In [220]:
train,test = train_test_split(df_portfolio1,test_size=0.3,shuffle=True)

In [221]:
idx_rec = recommender(train.drop('id',axis=1),df_features_enconding_std,model_knn)

In [226]:
calculate_accuracy(idx_rec,test['id'])

0.0